In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.output_parsers import StrOutputParser
import os
# from PyPDF2 import PdfReader
# from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.text_splitter import CharacterTextSplitter
# # from langchain.vectorstore
# from langchain_api.text_processing import TextChunker
# from langchain_api.embeddings.openai_tools import OpenAIEmbeds
# from langchain_api.vector_search import FAISSVectorStores import FAISS

# import os

In [ ]:
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
model = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

In [ ]:
# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)


In [ ]:
# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "test"}}

In [ ]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

In [ ]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a robot controller. You will give instructions to a jetbot to navigate to a dog based on {imageinput}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [ ]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    image: utf-8


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

## Potential

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_core.prompts import HumanMessagePromptTemplate, ChatPromptTemplate
from langchain_core.messages import HumanMessage

from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict
from langchain_core.messages import SystemMessage, trim_messages

import os

In [ ]:
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
model = ChatOpenAI(model="gpt-4o")

In [ ]:
trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)


In [ ]:
# image_path = 'path/to/image.png'
object = "dog"
detail_parameter = 'high'
messages=[AIMessage(content="I am in a field.")]


image_prompt_template = ChatPromptTemplate.from_messages(
    messages=[
        (
            "system",
            "You are a robot controller. You will give instructions to a jetbot to navigate to a {object} based on an image input on what the robot sees.",
        ),
        HumanMessage(content="These are the instructions you have given so far."),
        MessagesPlaceholder(variable_name="messages"),
        HumanMessage(content="""Describe the following image. 
                     If there is a {object} describe which direction the robot should travel to approach the {object}. 
                     Else, specify a direction the robot should move to find a new {object}."""),
        HumanMessagePromptTemplate.from_template(
            [{'image_url': {'url': '{image_path}', 'detail': '{detail_parameter}'}}]
        )
    ]
)
path = "imageofdog.png"
path2 = "http://[::]:8000/imageofdog.png"
prompt = image_prompt_template.format(image_path=path, detail_parameter=detail_parameter, object=object, messages=messages)

# print(prompt[:4000])

trimmed_messages = [AIMessage(content="I am still in a field.")]
prompt = image_prompt_template.invoke(
        {"messages": trimmed_messages, "object": "dog"}
)

print(prompt[:4000])

In [ ]:
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    object: str
    image: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = image_prompt_template.format(image_path=state["image"], detail_parameter=detail_parameter)
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "object": state["object"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc456"}}
query = "You are in a field"
object = "dog"
path = "imageofdog.png"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"object": object, "image": path},
    config,
)
output["messages"][-1].pretty_print()

## Main Loop

In [ ]:
from jetbot import Robot
import re

In [ ]:
def turn(args):
    try:
        angle = int(args)
        duration = abs(angle) / 90
        if angle > 0:
            robot.left(0.11)
        else:
            robot.right(0.11)
        time.sleep(duration)
        robot.stop()
        time.sleep(0.1)
        return f"Turned {angle} degrees."
    except ValueError:
        return "Invalid argument for turn. Please provide an integer."

def move_forward(args):
    try:
        duration = float(args)
        robot.forward(0.3)
        time.sleep(duration)
        robot.stop()
        time.sleep(0.1)

        return f"Moved forward for {duration} seconds."
    except ValueError:
        return "Invalid argument for move_forward. Please provide a number."

# <reasoning>.[<command><value>]
# ex:
# Center of the screen several meters away.[f3.5]
# The object was seen on the right side of the screen[t-30]

def parse_chatbot_response(response):
    """Parse chatbot response for reasoning and command."""
    match = re.search(r"(.+?)\[(f|t)([-\d.]+)]", response)
    reasoning = match.group(1).strip() if match else "No reasoning provided."
    command = match.group(2) if match else None
    value = match.group(3) if match else None
    return reasoning, command, value

def main():
    while(True):
        try:
            command, value = parse_chatbot_response(chatbot_response)
            chain.memory.add_memory_variables({"reasoning": reasoning})
            if command == "exit":
                break
            elif command == "f":
                print(move_forward(value))
                chain.memory.add_memory_variables({"commandvalue": f"Forward {value} seconds."})
            elif command == "t":
                print(turn(value))
                chain.memory.add_memory_variables({"commandvalue": f"Turn {value} degrees."})
            else:
                print("No valid instruction found.")

        camera.value 
        
        except Exception as e:
            print("Error executing command:", str(e))

In [ ]:
main()